## Author: Sidharth Ramanan
### Date: 05/28/22

Refer to each section of this notebook to do the 3 questions from the assignment. Feel free to make a copy of the notebook to run and explore the cells yourself (especially the random walk section), but while answering the questions, try to refer to the cell outputs from this copy since some results are stochastic/non-deterministic and consistent answers would be nice.


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
ratings_matrix = np.array([[4, None, 3.5, 2, 5],
                           [4.5, None, None, 3, 5],
                           [1, 2, 2, 4, 4.5],
                           [None, 3.5, 4, 4, None],
                           [None, 3, None, 5, None]
                           ])

In [ ]:
ratings_df = pd.DataFrame(ratings_matrix, columns=['P1', 'P2', 'P3', 'P4', 'P5'], index=['U1', 'U2', 'U3', 'U4', 'U5'])
ratings_df

,P1,P2,P3,P4,P5
U1,4,None,3.5,2,5
U2,4.5,None,None,3,5
U3,1,2,2,4,4.5
U4,None,3.5,4,4,None
U5,None,3,None,5,None


# Q1: User/User Collaborative Filtering

In [ ]:
item_ratings = ratings_df.fillna(ratings_df.mean())
item_ratings = item_ratings.T
item_ratings

,U1,U2,U3,U4,U5
P1,4.000000,4.500000,1.0,3.166667,3.166667
P2,2.833333,2.833333,2.0,3.500000,3.000000
P3,3.500000,3.166667,2.0,4.000000,3.166667
P4,2.000000,3.000000,4.0,4.000000,5.000000
P5,5.000000,5.000000,4.5,4.833333,4.833333


In [ ]:
user_ratings = ratings_df.T.fillna(ratings_df.T.mean())
user_ratings = user_ratings.T
user_ratings

,P1,P2,P3,P4,P5
U1,4.000000,3.625000,3.500000,2.0,5.000000
U2,4.500000,4.166667,4.166667,3.0,5.000000
U3,1.000000,2.000000,2.000000,4.0,4.500000
U4,3.833333,3.500000,4.000000,4.0,3.833333
U5,4.000000,3.000000,4.000000,5.0,4.000000


In [ ]:
user_ratings_centered = user_ratings.sub(user_ratings.mean(axis=1), axis=0)
user_ratings_centered

,P1,P2,P3,P4,P5
U1,0.375000,0.000000,-0.125000,-1.625000,1.375000
U2,0.333333,0.000000,0.000000,-1.166667,0.833333
U3,-1.700000,-0.700000,-0.700000,1.300000,1.800000
U4,0.000000,-0.333333,0.166667,0.166667,0.000000
U5,0.000000,-1.000000,0.000000,1.000000,0.000000


In [ ]:
item_ratings_centered = item_ratings.sub(item_ratings.mean(axis=1), axis=0)
item_ratings_centered

,U1,U2,U3,U4,U5
P1,0.833333,1.333333,-2.166667,0.000000,0.000000
P2,0.000000,0.000000,-0.833333,0.666667,0.166667
P3,0.333333,0.000000,-1.166667,0.833333,0.000000
P4,-1.600000,-0.600000,0.400000,0.400000,1.400000
P5,0.166667,0.166667,-0.333333,0.000000,0.000000


## User Similarities

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
user_similarities = pd.DataFrame(cosine_similarity(user_ratings_centered), columns=user_ratings_centered.index, index=user_ratings_centered.index)
user_similarities

,U1,U2,U3,U4,U5
U1,1.000000,0.993655,-0.029194,-0.329983,-0.530723
U2,0.993655,1.000000,-0.133592,-0.323575,-0.560449
U3,-0.029194,-0.133592,1.000000,0.275241,0.476731
U4,-0.329983,-0.323575,0.275241,1.000000,0.866025
U5,-0.530723,-0.560449,0.476731,0.866025,1.000000


## Item Similarities

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
item_similarities = pd.DataFrame(cosine_similarity(item_ratings_centered), columns=item_ratings_centered.index, index=item_ratings_centered.index)
item_similarities

,P1,P2,P3,P4,P5
P1,1.000000,0.624423,0.711974,-0.491429,0.991241
P2,0.624423,1.000000,0.960928,0.067666,0.629941
P3,0.711974,0.960928,1.000000,-0.198615,0.739600
P4,-0.491429,0.067666,-0.198615,1.000000,-0.537086
P5,0.991241,0.629941,0.739600,-0.537086,1.000000


# Q2: Matrix Factorization

In [ ]:
#Credit: https://towardsdatascience.com/recommendation-system-matrix-factorization-d61978660b4b

import numpy

def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    Q = Q.T

    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        eR = numpy.dot(P,Q)

        e = 0

        for i in range(len(R)):

            for j in range(len(R[i])):

                if R[i][j] > 0:

                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)

                    for k in range(K):

                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        # 0.001: local minimum
        if e < 0.001:

            break

    return P, Q.T

In [ ]:
ratings_df.fillna(0)

,P1,P2,P3,P4,P5
U1,4.0,0.0,3.5,2,5.0
U2,4.5,0.0,0.0,3,5.0
U3,1.0,2.0,2.0,4,4.5
U4,0.0,3.5,4.0,4,0.0
U5,0.0,3.0,0.0,5,0.0


In [ ]:
R = ratings_df.fillna(0).values
# N: num of User
N = len(R)
# M: num of Item
M = len(R[0])
# Num of Features
K = 3
 
U = np.random.rand(N,K)
V = np.random.rand(M,K)
 
U, V = matrix_factorization(R, U, V, K)

nR = np.dot(U, V.T)

In [ ]:
pd.DataFrame(U)

,0,1,2
0,0.178166,1.298959,1.629961
1,0.634324,1.215496,1.691221
2,1.065690,1.568855,-0.260529
3,1.318709,1.191822,1.484048
4,1.588832,1.496260,0.678211


In [ ]:
pd.DataFrame(V)

,0,1,2
0,0.324563,0.762423,1.893043
1,0.905440,0.752606,0.863758
2,0.803232,0.950854,1.244364
3,1.775453,1.372139,0.037560
4,1.017503,2.306852,1.015953


In [ ]:
pd.DataFrame(U @ V.T)

,0,1,2,3,4
0,4.133767,2.546816,3.406493,2.159900,4.833755
1,4.334153,2.949939,3.769763,2.857566,5.167598
2,1.048821,1.920614,2.023555,4.034984,4.438774
3,4.146041,3.372843,4.039173,4.032392,5.598869
4,2.940340,3.150497,3.542867,4.899447,5.757322


# Q3: Random Walks

In [ ]:
ratings_df

,P1,P2,P3,P4,P5
U1,4,None,3.5,2,5
U2,4.5,None,None,3,5
U3,1,2,2,4,4.5
U4,None,3.5,4,4,None
U5,None,3,None,5,None


In [ ]:
from collections import defaultdict
def constructGraph(ratingThreshold, ratings_df):
  Graph = defaultdict(list)
  users = ratings_df.index
  items = ratings_df.columns
  for user in users:
    for item in items:
      if ratings_df.loc[user][item] and ratings_df.loc[user][item] > ratingThreshold:
        Graph[user].append(item)
        Graph[item].append(user)
  return Graph

In [ ]:
ratingThreshold = 3
Graph = constructGraph(ratingThreshold, ratings_df)

In [ ]:
def randomWalk(Graph, N, user, jumpProb):
  scoreMap = defaultdict(int)
  for _ in range(N):
    currentNode = user
    while np.random.uniform() < jumpProb:
      currentNode = np.random.choice(Graph[currentNode])
    if "P" in currentNode:
      scoreMap[currentNode] += 1
  return scoreMap

In [ ]:
query = 'U5'
numWalks = 1000
jumpProb = 0.7
randomWalk(Graph, numWalks, query, jumpProb)

defaultdict(int, {'P1': 5, 'P2': 23, 'P3': 19, 'P4': 291, 'P5': 46})